In [8]:
import pandas as pd
import re
from openpyxl import load_workbook
from datetime import datetime, timedelta
import numpy as np

In [9]:
invoice = pd.read_excel('Invoice (2).xlsx')
invoice = invoice[~invoice["Invoice ID"].duplicated(keep=False)]
invoice = invoice[["Invoice Number", "Customer Name"]]

In [10]:
master = pd.read_excel('Copy of Master List of Customers_FY 25-26.xlsx',
                       skiprows=2,
                       sheet_name='FY 25-26')

master.columns = master.columns.str.strip()
master = master.drop(master.columns[10:], axis = True)
master = master.rename(columns={"Invoice": "Invoice Number"})

In [11]:
def extract_all(inv):
    nums = re.findall(r'\d+', str(inv))
    cleaned = [n[-3:] for n in nums if len(n) >= 3]
    return ['INV-25-26-000' + d for d in cleaned]

master_norm = master.copy()
master_norm['All Invoices'] = master_norm['Invoice Number'].apply(extract_all)
master_norm = master_norm.explode('All Invoices')

merged = invoice.merge(
    master_norm,
    left_on='Invoice Number',
    right_on='All Invoices',
    how='left'
)


In [12]:
mis = pd.read_excel('MIS_Coresphere100 Apr 25 to Oct 25.xlsx',
                    skiprows=2,
                    sheet_name='FY 25-26-Accrual')

mis = mis.rename(columns={"Invoice": "Invoice Number"})

In [13]:
TARGET_MONTH = 10
TARGET_YEAR = 2025

MONTH_NAMES = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
MONTH_NAME = MONTH_NAMES[TARGET_MONTH]

MIS_PATH = 'MIS_Coresphere100 Apr 25 to Oct 25.xlsx'
SHEET_NAME = 'FY 25-26-Accrual'

def get_month_start_end(month, year):
    start = datetime(year, month, 1)
    if month == 12:
        end = datetime(year, 12, 31)
    else:
        end = datetime(year, month + 1, 1) - timedelta(days=1)
    return start, end

def normalize_name(s):
    if pd.isna(s):
        return ""
    s = str(s).strip()
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", " ", s)
    return s.lower()

def parse_token_month_year(token):
    token = token.strip().lower().replace('.', '')
    m = re.search(r'([a-z]+)', token)
    y = re.search(r'(\d{2,4})', token)
    mm = None
    yy = None
    if m:
        key = m.group(1)[:3]
        mm = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,
              'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}.get(key)
    if y:
        val = y.group(1)
        yy = 2000 + int(val) if len(val) == 2 else int(val)
    return mm, yy

def months_list_from_field(v):
    if pd.isna(v):
        return []
    if isinstance(v, (pd.Timestamp, datetime)):
        return [v.month]
    s = str(v).strip().lower()
    if 'to' in s:
        parts = re.split(r'\s+to\s+', s)
        if len(parts) != 2:
            return []
        a, b = parts
        sm, sy = parse_token_month_year(a)
        em, ey = parse_token_month_year(b)
        if not sm or not em:
            return []
        if not sy:
            sy = TARGET_YEAR
        if not ey:
            ey = sy
        start = datetime(sy, sm, 1)
        end = datetime(ey, em, 28) + timedelta(days=4)
        end = end.replace(day=1) - timedelta(days=1)
        cur = start
        out = []
        while cur <= end:
            out.append(cur.month)
            ny = cur.year
            nm = cur.month + 1
            if nm == 13:
                nm = 1
                ny += 1
            cur = cur.replace(year=ny, month=nm, day=1)
        return out
    if '&' in s or ',' in s:
        tokens = re.split(r'[,&]', s)
        out = []
        for t in tokens:
            mm, yy = parse_token_month_year(t)
            if mm:
                out.append(mm)
        return sorted(set(out))
    m = re.findall(r'[A-Za-z]+', s)
    out = []
    for t in m:
        mm = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,
              'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}.get(t[:3])
        if mm:
            out.append(mm)
    return sorted(set(out))

def overlap_days(start, end, period_start, period_end):
    s = max(start, period_start)
    e = min(end, period_end)
    if e < s:
        return 0
    return (e - s).days + 1

def safe_round(x):
    if pd.isna(x) or x == '':
        return x
    try:
        return round(float(x), 3)
    except (ValueError, TypeError):
        return x

invoice = pd.read_excel('Invoice (2).xlsx')
invoice = invoice[~invoice["Invoice ID"].duplicated(keep=False)]
invoice = invoice[["Invoice Number", "Customer Name"]]

master = pd.read_excel('Copy of Master List of Customers_FY 25-26.xlsx',
                       skiprows=2, sheet_name='FY 25-26')
master.columns = master.columns.str.strip()
master = master.drop(master.columns[10:], axis=True)
master = master.rename(columns={"Invoice": "Invoice Number"})

def extract_all(inv):
    nums = re.findall(r'\d+', str(inv))
    cleaned = [n[-3:] for n in nums if len(n) >= 3]
    return ['INV-25-26-000' + d for d in cleaned]

master_norm = master.copy()
master_norm['All Invoices'] = master_norm['Invoice Number'].apply(extract_all)
master_norm = master_norm.explode('All Invoices')

merged = invoice.merge(
    master_norm,
    left_on='Invoice Number',
    right_on='All Invoices',
    how='left'
)

mis = pd.read_excel(MIS_PATH, skiprows=2, sheet_name=SHEET_NAME)
mis = mis.rename(columns=lambda x: x.strip() if isinstance(x, str) else x)

if 'Invoice' in mis.columns and 'Invoice Number' not in mis.columns:
    mis = mis.rename(columns={"Invoice": "Invoice Number"})

month_start, month_end = get_month_start_end(TARGET_MONTH, TARGET_YEAR)

mis['Customer Name'] = mis['Customer Name'].fillna('')
mis['cn_norm'] = mis['Customer Name'].apply(normalize_name)

for idx, row in mis.iterrows():
    start = pd.to_datetime(row.get('Start Date'), errors='coerce')
    end = pd.to_datetime(row.get('End Date'), errors='coerce')
    if pd.notna(start) and pd.notna(end):
        mis.at[idx, 'Contract period'] = (end - start).days + 1

merged = merged.copy()
if 'Customer Name_y' not in merged.columns and 'Customer Name' in merged.columns:
    merged = merged.rename(columns={'Customer Name': 'Customer Name_y'})
merged = merged.rename(columns=lambda x: x.strip() if isinstance(x, str) else x)
merged['Customer Name_y'] = merged['Customer Name_y'].fillna('')
merged['cn_norm'] = merged['Customer Name_y'].apply(normalize_name)

merged_map = {}
for _, r in merged.iterrows():
    merged_map.setdefault(r['cn_norm'], []).append(r)

col_name_days = f'Days in {MONTH_NAME}'
col_name_sales_days = f'{MONTH_NAME} Sales in days'
col_name_sales_months = f'{MONTH_NAME} Sales in months'

for idx, row in mis.iterrows():
    norm = row.get('cn_norm', '')
    start = pd.to_datetime(row.get('Start Date'), errors='coerce')
    end = pd.to_datetime(row.get('End Date'), errors='coerce')
    months_present = months_list_from_field(row.get('Months'))
    overlap = pd.notna(start) and pd.notna(end) and not (end < month_start or start > month_end)
    applicable = (TARGET_MONTH in months_present) or overlap
    
    if norm in merged_map:
        mrow = merged_map[norm][-1]
        inv_new = mrow.get('Invoice Number_y') or mrow.get('Invoice') or mrow.get('Invoice Number')
        inv_old = row.get('Invoice Number')
        
        if pd.isna(inv_old) or str(inv_old).strip() == "":
            if pd.notna(inv_new):
                mis.at[idx, 'Invoice Number'] = inv_new
        else:
            vals = [x.strip() for x in re.split(r'[;,]', str(inv_old)) if x.strip()]
            for p in re.split(r'[;,]', str(inv_new) if pd.notna(inv_new) else ''):
                p = p.strip()
                if p and p not in vals:
                    vals.append(p)
            mis.at[idx, 'Invoice Number'] = ", ".join(vals) if vals else inv_old
        
        pc_new = mrow.get('Payment Cycle')
        pc_old = row.get('Payment Cycle')
        if (pd.isna(pc_old) or str(pc_old).strip() == "") and pd.notna(pc_new):
            mis.at[idx, 'Payment Cycle'] = pc_new
        
        if pd.isna(start):
            start = pd.to_datetime(mrow.get('Start Date'), errors='coerce')
            if pd.notna(start):
                mis.at[idx, 'Start Date'] = start
        
        if pd.isna(end):
            end = pd.to_datetime(mrow.get('End Date'), errors='coerce')
            if pd.notna(end):
                mis.at[idx, 'End Date'] = end
    
    if applicable and pd.notna(start) and pd.notna(end):
        contract_period_days = (end - start).days + 1
        mis.at[idx, 'Contract period'] = contract_period_days
        d_month = overlap_days(start, end, month_start, month_end)
        amt = row.get('Contract Amount') if pd.notna(row.get('Contract Amount')) else 0
        sales_days = (amt / contract_period_days) * d_month if contract_period_days > 0 else 0
        
        months_diff = (end.year - start.year) * 12 + (end.month - start.month)
        if end.day >= start.day:
            mc = max(1, months_diff + 1)
        else:
            mc = max(1, months_diff)
        monthwise = amt / mc if mc > 0 else 0
        
        closing = (end.year == TARGET_YEAR and end.month == TARGET_MONTH)
        
        if TARGET_MONTH in months_present:
            sales_months = 0 if closing else monthwise
        elif overlap:
            sales_months = monthwise
        else:
            sales_months = np.nan
        
        mis.at[idx, col_name_days] = d_month
        mis.at[idx, col_name_sales_days] = sales_days
        if pd.notna(sales_months):
            mis.at[idx, col_name_sales_months] = sales_months

existing = set(mis['cn_norm'].fillna(''))
to_add = []

for _, r in merged.iterrows():
    nm = r['cn_norm']
    if nm in existing or nm == "":
        continue
    
    s = pd.to_datetime(r.get('Start Date'), errors='coerce')
    e = pd.to_datetime(r.get('End Date'), errors='coerce')
    overlap = pd.notna(s) and pd.notna(e) and not (e < month_start or s > month_end)
    
    if not overlap:
        continue
    
    new = pd.Series(index=mis.columns, dtype=object)
    if 'Months' in mis.columns:
        new['Months'] = f'{MONTH_NAME}-{str(TARGET_YEAR)[-2:]}'
    
    new['Customer Name'] = r.get('Customer Name_y')
    new['cn_norm'] = nm
    
    if 'Invoice Number' in mis.columns:
        inv_val = r.get('Invoice Number_y', r.get('Invoice Number', r.get('Invoice')))
        new['Invoice Number'] = inv_val
    
    if 'Nature of service' in mis.columns:
        new['Nature of service'] = r.get('Nature of service')
    
    new['Start Date'] = r.get('Start Date')
    new['End Date'] = r.get('End Date')
    new['Payment Cycle'] = r.get('Payment Cycle')
    new['Contract Amount'] = r.get('Contract Amount')
    
    if pd.notna(s) and pd.notna(e):
        contract_period_days = (e - s).days + 1
        new['Contract period'] = contract_period_days
        d_month = overlap_days(s, e, month_start, month_end)
        amt = new.get('Contract Amount') if pd.notna(new.get('Contract Amount')) else 0
        sales_days = (amt / contract_period_days) * d_month if contract_period_days > 0 else 0
        
        months_diff = (e.year - s.year) * 12 + (e.month - s.month)
        if e.day >= s.day:
            mc = max(1, months_diff + 1)
        else:
            mc = max(1, months_diff)
        monthwise = amt / mc if mc > 0 else 0
        
        closing = (e.year == TARGET_YEAR and e.month == TARGET_MONTH)
        
        if 'Months' in new.index:
            months_present_new = months_list_from_field(new.get('Months'))
        else:
            months_present_new = []
        
        if TARGET_MONTH in months_present_new:
            sales_months_new = 0 if closing else monthwise
        else:
            sales_months_new = monthwise
        
        new[col_name_days] = d_month
        new[col_name_sales_days] = sales_days
        new[col_name_sales_months] = sales_months_new
    
    to_add.append(new.to_dict())

if to_add:
    new_df = pd.DataFrame(to_add)
    for c in mis.columns:
        if c not in new_df.columns:
            new_df[c] = np.nan
    new_df = new_df[mis.columns]
    mis = pd.concat([mis, new_df], ignore_index=True)

amount_columns = ['Contract Amount', col_name_sales_days, col_name_sales_months]
for col in mis.columns:
    if any(keyword in str(col) for keyword in ['Sales', 'Invoiced amount', 'Days in']):
        if col not in amount_columns:
            amount_columns.append(col)

for c in amount_columns:
    if c in mis.columns:
        mis[c] = mis[c].apply(safe_round)

mis = mis.loc[:, ~mis.columns.duplicated()]
if 'cn_norm' in mis.columns:
    mis = mis.drop(columns=['cn_norm'])

mis = mis.rename(columns=lambda x: x.strip() if isinstance(x, str) else x)

for col in [col_name_days, col_name_sales_days, col_name_sales_months]:
    if col not in mis.columns:
        mis[col] = np.nan

wb = load_workbook(MIS_PATH)
ws = wb[SHEET_NAME]

header_row_excel = 3
max_row = ws.max_row
max_col = ws.max_column

for col_idx, col_name in enumerate(mis.columns, start=1):
    ws.cell(row=header_row_excel, column=col_idx).value = col_name

for row in range(header_row_excel + 1, max_row + 1):
    for col_idx in range(1, max_col + 1):
        ws.cell(row=row, column=col_idx).value = None

for r in range(mis.shape[0]):
    excel_row = header_row_excel + 1 + r
    for c_idx, col in enumerate(mis.columns, start=1):
        ws.cell(row=excel_row, column=c_idx).value = mis.iloc[r][col]

wb.save(MIS_PATH)

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

# File path
file_path = 'MIS_Coresphere100 Apr 25 to Oct 25.xlsx'

# Read the Excel file
print("Reading Excel file...")

# Read Invoices sheet (skip first 2 rows)
invoices_df = pd.read_excel(file_path, sheet_name='Invoices', skiprows=2)

# Read FY 25-26-Accrual sheet
accrual_df = pd.read_excel(file_path, sheet_name='FY 25-26-Accrual',skiprows=2)

print(f"Invoices sheet shape: {invoices_df.shape}")
print(f"Accrual sheet shape: {accrual_df.shape}")

# Extract B2B companies from Invoices sheet
print("\n=== Extracting B2B Companies ===")
b2b_companies = invoices_df[invoices_df['Nature'] == 'B2B']['Customer Name'].unique()
print(f"Total B2B companies found: {len(b2b_companies)}")
print("\nB2B Companies:")
for i, company in enumerate(b2b_companies, 1):
    print(f"{i}. {company}")

# Filter accrual data for B2B companies only
accrual_b2b_df = accrual_df[accrual_df['Customer Name'].isin(b2b_companies)].copy()

print(f"\n=== Accrual Data for B2B Companies ===")
print(f"Total records: {len(accrual_b2b_df)}")

# Convert date columns to datetime
accrual_b2b_df['Start Date'] = pd.to_datetime(accrual_b2b_df['Start Date'])
accrual_b2b_df['End Date'] = pd.to_datetime(accrual_b2b_df['End Date'])

# Extract month and year from End Date for monthly pivot
accrual_b2b_df['Month'] = accrual_b2b_df['End Date'].dt.to_period('M')
accrual_b2b_df['Month_Str'] = accrual_b2b_df['End Date'].dt.strftime('%b-%y')

# === MONTHLY PIVOT ===
print("\n=== Creating Monthly Pivot ===")

# Group by Payment Cycle and Month, sum the invoiced amounts
monthly_pivot = accrual_b2b_df.groupby(['Payment Cycle ', 'Month_Str']).agg({
    'Invoiced amount': 'sum'
}).reset_index()

# Pivot to wide format
monthly_pivot_wide = monthly_pivot.pivot(
    index='Payment Cycle',
    columns='Month_Str',
    values='Invoiced amount'
).fillna(0)

# Sort columns by date
month_order = pd.to_datetime(monthly_pivot_wide.columns, format='%b-%y').sort_values()
monthly_pivot_wide = monthly_pivot_wide[[dt.strftime('%b-%y') for dt in month_order]]

# Calculate totals
monthly_pivot_wide['Total'] = monthly_pivot_wide.sum(axis=1)

# Add row totals
total_row = monthly_pivot_wide.sum(axis=0)
total_row.name = 'Total'
monthly_pivot_wide = pd.concat([monthly_pivot_wide, total_row.to_frame().T])

print("\n=== Monthly Pivot (Payment Cycle vs Month) ===")
print(monthly_pivot_wide)

# === DATE-WISE PIVOT ===
print("\n=== Creating Date-wise Pivot ===")

# Create date column from End Date
accrual_b2b_df['Date'] = accrual_b2b_df['End Date'].dt.strftime('%m/%d/%Y')

# Group by Payment Cycle and Date
date_pivot = accrual_b2b_df.groupby(['Payment Cycle', 'Date']).agg({
    'Invoiced amount': 'sum'
}).reset_index()

# Pivot to wide format
date_pivot_wide = date_pivot.pivot(
    index='Payment Cycle',
    columns='Date',
    values='Invoiced amount'
).fillna(0)

# Sort columns by date
date_cols_sorted = pd.to_datetime(date_pivot_wide.columns).sort_values()
date_pivot_wide = date_pivot_wide[[dt.strftime('%m/%d/%Y') for dt in date_cols_sorted]]

# Calculate totals
date_pivot_wide['Total'] = date_pivot_wide.sum(axis=1)

# Add row totals
total_row_date = date_pivot_wide.sum(axis=0)
total_row_date.name = 'Total'
date_pivot_wide = pd.concat([date_pivot_wide, total_row_date.to_frame().T])

print("\n=== Date-wise Pivot (Payment Cycle vs Date) ===")
print(date_pivot_wide)

# === EXPORT TO EXCEL ===
print("\n=== Exporting to Excel ===")

output_file = 'B2B_Pivot_Analysis.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Write B2B companies list
    b2b_df = pd.DataFrame({'B2B Companies': b2b_companies})
    b2b_df.to_excel(writer, sheet_name='B2B Companies', index=False)
    
    # Write monthly pivot
    monthly_pivot_wide.to_excel(writer, sheet_name='Monthly Pivot')
    
    # Write date-wise pivot
    date_pivot_wide.to_excel(writer, sheet_name='Date-wise Pivot')
    
    # Write filtered B2B accrual data
    accrual_b2b_df.to_excel(writer, sheet_name='B2B Accrual Data', index=False)

print(f"\nOutput saved to: {output_file}")

# === SUMMARY STATISTICS ===
print("\n=== Summary Statistics ===")
print(f"Total B2B Companies: {len(b2b_companies)}")
print(f"Total B2B Transactions: {len(accrual_b2b_df)}")
print(f"Total B2B Invoiced Amount: ₹{accrual_b2b_df['Invoiced amount'].sum():,.2f}")
print(f"\nPayment Cycle Distribution:")
payment_cycle_summary = accrual_b2b_df.groupby('Payment Cycle').agg({
    'Invoiced amount': ['count', 'sum']
})
payment_cycle_summary.columns = ['Count', 'Total Amount']
print(payment_cycle_summary)

print("\n=== Script Completed Successfully ===")

Reading Excel file...
Invoices sheet shape: (742, 15)
Accrual sheet shape: (447, 31)

=== Extracting B2B Companies ===
Total B2B companies found: 359

B2B Companies:
1. Office of ASG (N. Venkatraman)
2. Kanungo & Co.
3. Diwan Law Associates
4. Aditya Sikka / Tvashta Law
5. Amrita Panda / Chambers of Amrita Panda
6. Ish Jain / Regius Partners LLP
7. The Guild | Advocates & Associate Counsel
8. Manish kelker
9. Abhimanyu Garg / Methodica Legal
10. SRCO Legal / Salvador Raghav & Co.
11. Agama Law Associates
12. Gourab Banerji
13. Parakram Legal
14. Mustafa S. Doctor
15. Keystone Partners Advocates & Solicitors
16. Naushad Engineer
17. Farhan Dubash
18. Venkatesh Dhond
19. Shyam Kapadia
20. Gaurav Joshi
21. Prantar
22. Sajal Awasthi
23. Shiraz Rustomjee
24. Tejas Bhide
25. Saswat Mohanty
26. ANG Partners
27. Asahi Legal LLP
28. Vesta Legal
29. Jehangir Jejeebhoy
30. Darius J. Khambata
31. Abel Tom Benny // Thomas and Thomas Advocates
32. MZM Legal LLP
33. Prompt Legal
34. Monalisa Kosaria


KeyError: 'Payment Cycle '